In [1]:
import findspark
findspark.init()

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, Row
from random import random
from operator import add
import time

In [2]:
def f(_):
    x = random() * 2 - 1
    y = random() * 2 - 1
    return 1 if x ** 2 + y ** 2 <= 1 else 0

In [3]:
%env MESOS_NATIVE_JAVA_LIBRARY=/usr/local/lib/libmesos.so

env: MESOS_NATIVE_JAVA_LIBRARY=/usr/local/lib/libmesos.so


In [4]:
start_time = time.time()

spark = SparkSession.builder\
        .appName("SparkPiParallel")\
        .master("mesos://zk://10.129.64.20:2181,10.129.64.10:2181,10.129.64.30:2181/mesos") \
        .config("spark.executor.uri", "http://lasid20/spark-2.4.4-bin-hadoop2.7.tgz")\
        .getOrCreate()

spark.conf.set("spark.submit.deployMode", "cluster")
spark.conf.set("spark.driver.supervise", "true")
spark.conf.set("spark.executor.memory", "8g")
spark.conf.set("spark.core.connection.ack.wait.timeout", "12000")

In [5]:
partitions = 200
n = 100000 * partitions

count = spark.sparkContext.parallelize(range(1, n + 1), partitions).map(f).reduce(add)

print("Pi is roughly %f" % (4.0 * count / n))
print("--- Parallel Execution time: %s seconds ---" % (time.time() - start_time))

Pi is roughly 3.150000
--- Parallel Execution time: 12.813469409942627 seconds ---


In [6]:
spark.stop()